In [1]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats

client=pymongo.MongoClient("192.168.10.240",27017)
db= client['AI_LKJ']
collection_9suk = db['kto9suk9suk_review']
collection_yeomi = db['yeomi_review']

data_consume_9suk = collection_9suk.find()
data_consume_yeomi = collection_yeomi.find()

df_consume_9suk = pd.DataFrame(data_consume_9suk)
df_consume_yeomi = pd.DataFrame(data_consume_yeomi)

### 데이터셋

In [2]:
# 텍스트 파일 경로
file_path = 'steam.txt'

# 텍스트 파일 읽기
with open(file_path, 'r') as file:
    read_data = file.read()

In [3]:
import pandas as pd
from io import StringIO

# 텍스트 데이터를 파일 객체로 변환
file_obj = StringIO(read_data)
file_obj
# 데이터프레임 생성
df_labeled = pd.read_csv(file_obj, sep='\t')
df_labeled

,0,노래가 너무 적음
0,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다."
1,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
2,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??
3,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요
4,1,역시 재미있네요 전작에서 할수 없었던 자유로운 덱 빌딩도 좋네요^^
...,...,...
99994,0,한글화해주면 10개산다
99995,0,개쌉노잼 ㅋㅋ
99996,0,노잼이네요... 30분하고 지웠어요...
99997,1,야생을 사랑하는 사람들을 위한 짧지만 여운이 남는 이야기. 영어는 그리 어렵지 않습니다.


In [4]:
df_labeled.columns = ["sentiment","comment"]
new_record = {'sentiment': 0, 'comment': "노래가 너무 적음"}
df_labeled = pd.concat([df_labeled, pd.DataFrame([new_record])], ignore_index=True)
df_labeled

,sentiment,comment
0,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다."
1,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
2,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??
3,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요
4,1,역시 재미있네요 전작에서 할수 없었던 자유로운 덱 빌딩도 좋네요^^
...,...,...
99995,0,개쌉노잼 ㅋㅋ
99996,0,노잼이네요... 30분하고 지웠어요...
99997,1,야생을 사랑하는 사람들을 위한 짧지만 여운이 남는 이야기. 영어는 그리 어렵지 않습니다.
99998,1,한국의 메탈레이지를 떠오르게한다 진짜 손맛으로 하는게임


### 피클 불러오기

In [5]:
import pickle

# 피클 파일에서 객체 불러오기
with open('basic_modeling.pickle', 'rb') as f:
    logistic_model = pickle.load(f)
    svm_model = pickle.load(f)
    
# 불러온 객체 출력
print(logistic_model)
print(svm_model)

LogisticRegression()
SVC()


### 머신러닝 학습

#### 훈련용,시험용 데이터셋 설정

In [6]:
from sklearn.model_selection import train_test_split
labeled_x = df_labeled['comment']
labeled_y = df_labeled['sentiment']
labeled_x_train,labeled_x_test = train_test_split(labeled_x,test_size=0.3,random_state=42)
labeled_y_train,labeled_y_test = train_test_split(labeled_y,test_size=0.3,random_state=42)

In [7]:
labeled_x_test

75721    짧고 굵다 좋은 아이디어를 가졋음에도 플레이타임에 집착하느라 질질 끌어서 게임 자체...
80184    분위기 깡패. 플레이 하는 사람 따라서 심리적으로 힘들수도 있을것 같습니다. 살인,...
19864    단순하고 심플하지만 사람 짜증나게 하고 몇번씩 종료 시키게 만드는 뭐 같은 게임의욕...
76699                                          오우 바니걸 먹는다!
92991    3300원이 아깝지 않다. 방패로 싸우고 활로 싸우고 검으로 싸우고 총으로 싸우고아...
                               ...                        
42648                                               돈장난이 좀
86306                         스토리 텔링이 너무 마음에 들었어요! 수작! 강추!
45466    스토리 몰입도가 엄청나서 게임킬때마다 켠김왕을 하게 되어버린다. 이제 2회차째 '-...
63724    길은 드럽게 복잡함 그래픽도 아웃라스트보다 떨어짐 갑툭튀도 없고 쫓기는 씬도 없고 ...
34122    '이시대 최고의 망겜' 레벨이 8까지 있는데 운나쁘면 버그생겨서 못깸. 무료로 받아...
Name: comment, Length: 30000, dtype: object

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfVectorizer = TfidfVectorizer(use_idf=True,ngram_range=(1,2))
labeled_x_train = tfidfVectorizer.fit_transform(labeled_x_train)
labeled_x_test = tfidfVectorizer.transform(labeled_x_test)
dictionary_list = tfidfVectorizer.get_feature_names_out()
dictionary_list

array(['00', '00 12', '000', ...,
       '９年も前のゲームにハードモードが追加されてた ホラーゲームの中で一番好きな作品だから嬉しい', 'ｚｌｑｈｊd아니시발',
       'ｚｌｑｈｊd아니시발 키보드설정이'], dtype=object)

#### 로지스틱 회귀분석

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris

# 탐색할 하이퍼파라미터 설정
param_grid = {'C': [0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

# 그리드 서치를 사용하여 최적의 하이퍼파라미터 탐색
grid_search = GridSearchCV(logistic_model, param_grid, cv=5)
grid_search.fit(labeled_x_train, labeled_y_train)

# 최적의 하이퍼파라미터 출력
print("최적 하이퍼파라미터:", grid_search.best_params_)

# 최적의 모델로 예측
logistic_best_model = grid_search.best_estimator_         

/usr/local/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py", line 1172, in fit
    solver = _check_solver

최적 하이퍼파라미터: {'C': 10, 'penalty': 'l2'}


In [10]:
labeled_y_test_predict = logistic_best_model.predict(labeled_x_test)
from sklearn.metrics import classification_report
print(classification_report(labeled_y_test, labeled_y_test_predict))

              precision    recall  f1-score   support

           0       0.78      0.75      0.76     15161
           1       0.75      0.78      0.77     14839

    accuracy                           0.77     30000
   macro avg       0.77      0.77      0.77     30000
weighted avg       0.77      0.77      0.77     30000



#### SVM

In [11]:
# from sklearn import svm
# from sklearn.model_selection import GridSearchCV
# from sklearn.datasets import load_iris

# # 탐색할 하이퍼파라미터 설정
# param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf', 'linear']}

# # 그리드 서치를 사용하여 최적의 하이퍼파라미터 탐색
# grid_search = GridSearchCV(svm_model, param_grid, cv=5)
# grid_search.fit(labeled_x_train, labeled_y_train)

# # 최적의 하이퍼파라미터 출력
# print("최적 하이퍼파라미터:", grid_search.best_params_)

# # 최적의 모델로 예측
# svm_best_model = grid_search.best_estimator_    

In [12]:
# labeled_y_test_predict = svm_best_model.predict(labeled_x_test)
# from sklearn.metrics import classification_report
# print(classification_report(labeled_y_test, labeled_y_test_predict))

#### Hyper parameter 적용 모델 결과

#### 피클 내보내기

In [13]:
import pickle

# 내보낼 객체
data = {'example': 'data'}

# 객체를 피클 파일로 내보내기
with open('hyper_modeling.pickle', 'wb') as f:
    pickle.dump(logistic_best_model, f)

#### 참조페이지

https://github.com/bab2min/corpus/tree/master/sentiment